In [1]:
!pip install beautifulsoup4
!pip install requests

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Set up headless browser
options = Options()
options.headless = True  # Run in the background without opening a window
driver = webdriver.Chrome(options=options)

# Open the target page
url = "https://www.coldwellbanker.com/find-agents"
driver.get(url)

# Get the page source and print it
html = driver.page_source
print(html)

# Close the browser
driver.quit()

<html lang="en-US"><head><style data-emotion="css-global" data-s="">html{-webkit-font-smoothing:antialiased;-moz-osx-font-smoothing:grayscale;box-sizing:border-box;-webkit-text-size-adjust:100%;}*,*::before,*::after{box-sizing:inherit;}strong,b{font-weight:700;}body{margin:0;color:rgba(0, 0, 0, 0.87);font-family:Roboto;font-weight:400;font-size:16px;line-height:24px;letter-spacing:0.8px;background-color:#FFFFFF;}@media (max-width:599.95px){body{font-size:16px;line-height:24px;letter-spacing:0.4px;}}@media print{body{background-color:#fff;}}body::backdrop{background-color:#FFFFFF;}h1,h2,h3,h4,h5,h6,p{margin:0;}</style><style data-emotion="css-global" data-s=""></style><style data-emotion="css-global" data-s=""></style><style data-emotion="css 1rr9cau-MuiBackdrop-root xdbj64-MuiCircularProgress-root 1idz92c-MuiCircularProgress-svg 176wh8e-MuiCircularProgress-circle wkb4so 4amycm-MuiPaper-root-MuiAppBar-root 1ed3hv0-MuiContainer-root 1iezxgl-MuiToolbar-root vpk1t2 5akf41-MuiButtonBase-roo

In [5]:
!pip install selenium webdriver-manager

In [7]:
!pip install undetected-chromedriver

In [9]:
import time
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def scrape_coldwell_agents(url):
    # Set up undetected Chrome (avoids bot blocks)
    options = uc.ChromeOptions()
    options.headless = False  # Set to True after testing
    options.add_argument("--window-size=1920,1080")
    
    driver = uc.Chrome(options=options)
    driver.get(url)
    
    # Wait for agent cards to load
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.MuiGrid-item.css-mgehqq'))
        )
    except:
        print("Timed out waiting for page to load")
        driver.quit()
        return []
    
    # Scroll to load all agents (if lazy-loaded)
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    # Parse the page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    agents = []
    
    for card in soup.select('div.MuiGrid-item.css-mgehqq'):
        try:
            name = card.select_one('h1.MuiTypography-subtitle2').text.strip()
            
            # Phone extraction with fallbacks
            phone_div = card.find('div', {'data-testid': 'phoneNumber'})
            if phone_div:
                phone = phone_div.find('a', {'data-testid': 'phoneLink'})
                phone = phone.text.strip() if phone else "Phone not found"
            else:
                phone = "Phone not found"
            
            agents.append({
                'name': name,
                'phone': phone,
            })
        except Exception as e:
            print(f"Error parsing card: {e}")
            continue
    
    driver.quit()
    return agents

# Example usage
if __name__ == "__main__":
    target_url = "https://www.coldwellbanker.com/zip/tx/75001/agents"
    agents = scrape_coldwell_agents(target_url)
    
    print(f"\nFound {len(agents)} agents:")
    for idx, agent in enumerate(agents, 1):
        print(f"{idx}. {agent['name']} - {agent['phone']}")


Found 12 agents:
1. Judith Abbott - (214) 228-9828
2. Venesa Acosta - (817) 683-0665
3. Julie Adler - (214) 223-7255
4. Sara Aldiab - (817) 937-7736
5. Atila Almeida - (972) 802-2256
6. Jessica Amber - (951) 675-0909
7. Mary Anderson - (214) 694-4633
8. Alessandra Antonio - (972) 652-0139
9. BJ Antweil - (214) 237-4500
10. Kenneth Arenas - (469) 826-9788
11. Cesar Arevalo Martinez - (972) 800-0809
12. Myra Baginski - (214) 918-1765


In [19]:
import time
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def scrape_coldwell_agents(url, max_pages=None):
    # Set up undetected Chrome (avoids bot blocks)
    options = uc.ChromeOptions()
    options.headless = False  # Set to True after testing
    options.add_argument("--window-size=1920,1080")
    
    driver = uc.Chrome(options=options)
    driver.get(url)
    
    all_agents = []
    page_count = 1
    
    while True:
        # Wait for agent cards to load
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.MuiGrid-item.css-mgehqq'))
            )
        except:
            print("Timed out waiting for page to load")
            break
        
        # Scroll to load all agents (if lazy-loaded)
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        # Parse the page
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        agents = []
        
        for card in soup.select('div.MuiGrid-item.css-mgehqq'):
            try:
                name = card.select_one('h1.MuiTypography-subtitle2').text.strip()
                
                # Phone extraction with fallbacks
                phone_div = card.find('div', {'data-testid': 'phoneNumber'})
                if phone_div:
                    phone = phone_div.find('a', {'data-testid': 'phoneLink'})
                    phone = phone.text.strip() if phone else "Phone not found"
                else:
                    phone = "Phone not found"
                
                agents.append({
                    'name': name,
                    'phone': phone,
                })
            except Exception as e:
                print(f"Error parsing card: {e}")
                continue
        
        all_agents.extend(agents)
        print(f"Scraped page {page_count} - {len(agents)} agents found")
        
        # Check if we've reached the max pages requested
        if max_pages and page_count >= max_pages:
            break
            
        # Try to find and click the next page button
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Go to next page"]'))
            )
            
            # Check if button is disabled (no more pages)
            if "Mui-disabled" in next_button.get_attribute("class"):
                print("Reached last page")
                break
                
            next_button.click()
            page_count += 1
            time.sleep(3)  # Wait for next page to load
        except Exception as e:
            print(f"Could not find next page button or error navigating: {e}")
            break
    
    driver.quit()
    return all_agents

# Example usage
if __name__ == "__main__":
    target_url = "https://www.coldwellbanker.com/zip/tx/75001/agents"
    agents = scrape_coldwell_agents(target_url, max_pages=3)  # Set max_pages=None to scrape all pages
    
    print(f"\nFound {len(agents)} agents in total:")
    for idx, agent in enumerate(agents, 1):
        print(f"{idx}. {agent['name']} - {agent['phone']}")

Scraped page 1 - 12 agents found
Could not find next page button or error navigating: Message: element click intercepted: Element <button class="MuiButtonBase-root MuiPaginationItem-root MuiPaginationItem-sizeLarge MuiPaginationItem-text MuiPaginationItem-rounded MuiPaginationItem-textPrimary MuiPaginationItem-previousNext css-hnwmdn" tabindex="0" type="button" aria-label="Go to next page">...</button> is not clickable at point (603, 20). Other element would receive the click: <button class="MuiButtonBase-root MuiButton-root MuiButton-text MuiButton-textPrimary MuiButton-sizeMedium MuiButton-textSizeMedium MuiButton-root MuiButton-text MuiButton-textPrimary MuiButton-sizeMedium MuiButton-textSizeMedium css-q2y5p0-MuiButtonBase-root-MuiButton-root" tabindex="0" type="button" id="Agents &amp; Offices" data-testid="header-customized-test" aria-haspopup="true">...</button>
  (Session info: chrome=135.0.7049.114)
Stacktrace:
	GetHandleVerifier [0x0032D363+60275]
	GetHandleVerifier [0x0032D3

In [25]:
import time
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import (ElementClickInterceptedException, 
                                      TimeoutException,
                                      NoSuchElementException)

def scrape_coldwell_agents(url, max_pages=None):
    # Set up undetected Chrome
    options = uc.ChromeOptions()
    options.headless = False  # Set to True for production
    options.add_argument("--window-size=1920,1080")
    
    driver = uc.Chrome(options=options)
    driver.get(url)
    
    all_agents = []
    page_count = 1
    
    while True:
        # Wait for agent cards to load
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.MuiGrid-item.css-mgehqq'))
            )
        except TimeoutException:
            print("Timed out waiting for page to load")
            break
        
        # Scroll to load all agents
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        # Parse the page
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        agents = []
        
        for card in soup.select('div.MuiGrid-item.css-mgehqq'):
            try:
                name = card.select_one('h1.MuiTypography-subtitle2').text.strip()
                
                # Phone extraction with fallbacks
                phone_div = card.find('div', {'data-testid': 'phoneNumber'})
                if phone_div:
                    phone = phone_div.find('a', {'data-testid': 'phoneLink'})
                    phone = phone.text.strip() if phone else "Phone not found"
                else:
                    phone = "Phone not found"
                
                agents.append({
                    'name': name,
                    'phone': phone,
                })
            except Exception as e:
                print(f"Error parsing card: {e}")
                continue
        
        all_agents.extend(agents)
        print(f"Scraped page {page_count} - {len(agents)} agents found")
        
        # Check if we've reached the max pages requested
        if max_pages and page_count >= max_pages:
            break
            
        # Enhanced pagination handling
        try:
            # Scroll to top to avoid header interference
            driver.execute_script("window.scrollTo(0, 0)")
            time.sleep(1)
            
            # Try to find the next page button
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Go to next page"]')
            except NoSuchElementException:
                print("No more pages available")
                break
                
            # Check if button is disabled
            if "Mui-disabled" in next_button.get_attribute("class"):
                print("Reached last page")
                break
                
            # Multiple attempts to click the button
            for attempt in range(3):
                try:
                    # Scroll the button into view
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    time.sleep(1)
                    
                    # Try regular click first
                    next_button.click()
                    print("Clicked next button normally")
                    break
                except ElementClickInterceptedException:
                    # Fallback to JavaScript click
                    driver.execute_script("arguments[0].click();", next_button)
                    print("Used JavaScript click fallback")
                    break
                except Exception as e:
                    if attempt == 2:
                        raise
                    print(f"Click attempt {attempt + 1} failed, retrying...")
                    time.sleep(2)
            
            # Wait for page to load
            try:
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.MuiGrid-item.css-mgehqq'))
                )
                page_count += 1
                time.sleep(2)  # Additional delay for stability
            except TimeoutException:
                print("New page didn't load properly")
                break
                
        except Exception as e:
            print(f"Navigation error: {str(e).split('Stacktrace:')[0]}")
            break
    
    driver.quit()
    return all_agents

# Example usage
if __name__ == "__main__":
    target_url = "https://www.coldwellbanker.com/zip/tx/75001/agents"
    agents = scrape_coldwell_agents(target_url, max_pages=3)
    
    print(f"\nFound {len(agents)} agents in total:")
    for idx, agent in enumerate(agents, 1):
        print(f"{idx}. {agent['name']} - {agent['phone']}")

Scraped page 1 - 12 agents found
Clicked next button normally
Scraped page 2 - 12 agents found
Clicked next button normally
Scraped page 3 - 12 agents found

Found 36 agents in total:
1. Judith Abbott - (214) 228-9828
2. Venesa Acosta - (817) 683-0665
3. Julie Adler - (214) 223-7255
4. Sara Aldiab - (817) 937-7736
5. Atila Almeida - (972) 802-2256
6. Jessica Amber - (951) 675-0909
7. Mary Anderson - (214) 694-4633
8. Alessandra Antonio - (972) 652-0139
9. BJ Antweil - (214) 237-4500
10. Kenneth Arenas - (469) 826-9788
11. Cesar Arevalo Martinez - (972) 800-0809
12. Myra Baginski - (214) 918-1765
13. Shital Balar - (214) 237-4500
14. Shirin Baldwin - (972) 482-4611
15. Erin Bardin - (214) 796-1655
16. Courtney Barrett - (214) 727-6206
17. Mylea Beam - (214) 534-5038
18. Colt Bell - (940) 435-5336
19. Jennifer Bencomo - (972) 598-3639
20. Sebastian Benitez - (469) 258-4433
21. Todd Berther - (214) 264-6149
22. Todd Blersch - (513) 604-3138
23. Ben Blum - (214) 237-4500
24. Marie Boudriau

In [27]:
import time
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import (ElementClickInterceptedException, 
                                      TimeoutException,
                                      NoSuchElementException)

def scrape_coldwell_agents(url):
    # Set up undetected Chrome
    options = uc.ChromeOptions()
    options.headless = False  # Set to True for production
    options.add_argument("--window-size=1920,1080")
    
    driver = uc.Chrome(options=options)
    driver.get(url)
    
    all_agents = []
    page_count = 1
    
    while True:
        # Wait for agent cards to load
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.MuiGrid-item.css-mgehqq'))
            )
        except TimeoutException:
            print("Timed out waiting for page to load")
            break
        
        # Scroll to load all agents
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        # Parse the page
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        agents = []
        
        for card in soup.select('div.MuiGrid-item.css-mgehqq'):
            try:
                name = card.select_one('h1.MuiTypography-subtitle2').text.strip()
                
                # Phone extraction with fallbacks
                phone_div = card.find('div', {'data-testid': 'phoneNumber'})
                if phone_div:
                    phone = phone_div.find('a', {'data-testid': 'phoneLink'})
                    phone = phone.text.strip() if phone else "Phone not found"
                else:
                    phone = "Phone not found"
                
                agents.append({
                    'name': name,
                    'phone': phone,
                })
            except Exception as e:
                print(f"Error parsing card: {e}")
                continue
        
        all_agents.extend(agents)
        print(f"Scraped page {page_count} - {len(agents)} agents found")
            
        # Enhanced pagination handling
        try:
            # Scroll to top to avoid header interference
            driver.execute_script("window.scrollTo(0, 0)")
            time.sleep(1)
            
            # Try to find the next page button
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Go to next page"]')
            except NoSuchElementException:
                print("No more pages available")
                break
                
            # Check if button is disabled
            if "Mui-disabled" in next_button.get_attribute("class"):
                print("Reached last page")
                break
                
            # Multiple attempts to click the button
            for attempt in range(3):
                try:
                    # Scroll the button into view
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    time.sleep(1)
                    
                    # Try regular click first
                    next_button.click()
                    print("Clicked next button normally")
                    break
                except ElementClickInterceptedException:
                    # Fallback to JavaScript click
                    driver.execute_script("arguments[0].click();", next_button)
                    print("Used JavaScript click fallback")
                    break
                except Exception as e:
                    if attempt == 2:
                        print("Failed to click next button after 3 attempts")
                        raise
                    print(f"Click attempt {attempt + 1} failed, retrying...")
                    time.sleep(2)
            
            # Wait for page to load
            try:
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.MuiGrid-item.css-mgehqq'))
                )
                page_count += 1
                time.sleep(2)  # Additional delay for stability
            except TimeoutException:
                print("New page didn't load properly")
                break
                
        except Exception as e:
            print(f"Navigation error: {str(e).split('Stacktrace:')[0]}")
            break
    
    driver.quit()
    return all_agents

# Example usage
if __name__ == "__main__":
    target_url = "https://www.coldwellbanker.com/zip/tx/75001/agents"
    print("Starting scraping...")
    agents = scrape_coldwell_agents(target_url)
    
    print(f"\nFound {len(agents)} agents in total:")
    for idx, agent in enumerate(agents, 1):
        print(f"{idx}. {agent['name']} - {agent['phone']}")

Starting scraping...
Scraped page 1 - 12 agents found
Clicked next button normally
Scraped page 2 - 12 agents found
Clicked next button normally
Scraped page 3 - 12 agents found
Clicked next button normally
Scraped page 4 - 12 agents found
Clicked next button normally
Scraped page 5 - 12 agents found
Clicked next button normally
Scraped page 6 - 12 agents found
Clicked next button normally
Scraped page 7 - 12 agents found
Clicked next button normally
Scraped page 8 - 12 agents found
Clicked next button normally
Scraped page 9 - 12 agents found
Clicked next button normally
Scraped page 10 - 12 agents found
Clicked next button normally
Scraped page 11 - 12 agents found
Clicked next button normally
Scraped page 12 - 12 agents found
Clicked next button normally
Scraped page 13 - 12 agents found
Clicked next button normally
Scraped page 14 - 12 agents found
Clicked next button normally
Scraped page 15 - 12 agents found
Clicked next button normally
Scraped page 16 - 12 agents found
Clicked n

In [29]:
import time
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import (ElementClickInterceptedException, 
                                      TimeoutException,
                                      NoSuchElementException)

def scrape_coldwell_agents_for_zip(zip_code):
    url = f"https://www.coldwellbanker.com/zip/tx/{zip_code}/agents"
    print(f"\nStarting scraping for ZIP code: {zip_code}")
    
    # Set up undetected Chrome
    options = uc.ChromeOptions()
    options.headless = False  # Set to True for production
    options.add_argument("--window-size=1920,1080")
    
    driver = uc.Chrome(options=options)
    driver.get(url)
    
    all_agents = []
    page_count = 1
    
    while True:
        # Wait for agent cards to load
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.MuiGrid-item.css-mgehqq'))
            )
        except TimeoutException:
            print(f"Timed out waiting for page to load for ZIP {zip_code}")
            break
        
        # Scroll to load all agents
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        # Parse the page
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        agents = []
        
        for card in soup.select('div.MuiGrid-item.css-mgehqq'):
            try:
                name = card.select_one('h1.MuiTypography-subtitle2').text.strip()
                
                # Phone extraction with fallbacks
                phone_div = card.find('div', {'data-testid': 'phoneNumber'})
                if phone_div:
                    phone = phone_div.find('a', {'data-testid': 'phoneLink'})
                    phone = phone.text.strip() if phone else "Phone not found"
                else:
                    phone = "Phone not found"
                
                agents.append({
                    'name': name,
                    'phone': phone,
                    'zip_code': zip_code  # Add ZIP code to agent record
                })
            except Exception as e:
                print(f"Error parsing card: {e}")
                continue
        
        all_agents.extend(agents)
        print(f"Scraped page {page_count} for ZIP {zip_code} - {len(agents)} agents found")
            
        # Enhanced pagination handling
        try:
            # Scroll to top to avoid header interference
            driver.execute_script("window.scrollTo(0, 0)")
            time.sleep(1)
            
            # Try to find the next page button
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Go to next page"]')
            except NoSuchElementException:
                print(f"No more pages available for ZIP {zip_code}")
                break
                
            # Check if button is disabled
            if "Mui-disabled" in next_button.get_attribute("class"):
                print(f"Reached last page for ZIP {zip_code}")
                break
                
            # Multiple attempts to click the button
            for attempt in range(3):
                try:
                    # Scroll the button into view
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    time.sleep(1)
                    
                    # Try regular click first
                    next_button.click()
                    print("Clicked next button normally")
                    break
                except ElementClickInterceptedException:
                    # Fallback to JavaScript click
                    driver.execute_script("arguments[0].click();", next_button)
                    print("Used JavaScript click fallback")
                    break
                except Exception as e:
                    if attempt == 2:
                        print("Failed to click next button after 3 attempts")
                        raise
                    print(f"Click attempt {attempt + 1} failed, retrying...")
                    time.sleep(2)
            
            # Wait for page to load
            try:
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.MuiGrid-item.css-mgehqq'))
                )
                page_count += 1
                time.sleep(2)  # Additional delay for stability
            except TimeoutException:
                print("New page didn't load properly")
                break
                
        except Exception as e:
            print(f"Navigation error: {str(e).split('Stacktrace:')[0]}")
            break
    
    driver.quit()
    return all_agents

def scrape_multiple_zips(zip_codes):
    all_agents = []
    for zip_code in zip_codes:
        agents = scrape_coldwell_agents_for_zip(zip_code)
        all_agents.extend(agents)
        print(f"\nFinished scraping ZIP {zip_code}. Total agents so far: {len(all_agents)}")
        time.sleep(2)  # Brief pause between ZIP codes
    
    return all_agents

# Example usage
if __name__ == "__main__":
    zip_codes = ['77494', '77449', '78660', '77433']
    print("Starting scraping for multiple ZIP codes...")
    all_agents = scrape_multiple_zips(zip_codes)
    
    print(f"\nFinal Results: Found {len(all_agents)} agents across all ZIP codes:")
    for idx, agent in enumerate(all_agents, 1):
        print(f"{idx}. {agent['name']} - {agent['phone']} (ZIP: {agent['zip_code']})")

Starting scraping for multiple ZIP codes...

Starting scraping for ZIP code: 77494
Scraped page 1 for ZIP 77494 - 12 agents found
Clicked next button normally
Scraped page 2 for ZIP 77494 - 12 agents found
Clicked next button normally
Scraped page 3 for ZIP 77494 - 12 agents found
Clicked next button normally
Scraped page 4 for ZIP 77494 - 12 agents found
Clicked next button normally
Scraped page 5 for ZIP 77494 - 12 agents found
Clicked next button normally
Scraped page 6 for ZIP 77494 - 12 agents found
Clicked next button normally
Scraped page 7 for ZIP 77494 - 12 agents found
Clicked next button normally
Scraped page 8 for ZIP 77494 - 12 agents found
Clicked next button normally
Scraped page 9 for ZIP 77494 - 12 agents found
Clicked next button normally
Scraped page 10 for ZIP 77494 - 12 agents found
Clicked next button normally
Scraped page 11 for ZIP 77494 - 12 agents found
Clicked next button normally
Scraped page 12 for ZIP 77494 - 12 agents found
Clicked next button normally
Sc

KeyboardInterrupt: 

In [31]:
import time
import csv
import os
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import (ElementClickInterceptedException, 
                                      TimeoutException,
                                      NoSuchElementException)

CSV_FILE = "coldwell_agents.csv"

def save_agents_to_csv(agents, mode='a'):
    file_exists = os.path.isfile(CSV_FILE)
    with open(CSV_FILE, mode, newline='', encoding='utf-8') as csvfile:
        fieldnames = ['name', 'phone', 'zip_code']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if mode == 'w' or not file_exists:
            writer.writeheader()
        for agent in agents:
            writer.writerow(agent)

def scrape_coldwell_agents_for_zip(zip_code):
    url = f"https://www.coldwellbanker.com/zip/tx/{zip_code}/agents"
    print(f"\nStarting scraping for ZIP code: {zip_code}")
    
    # Set up undetected Chrome
    options = uc.ChromeOptions()
    options.headless = False
    options.add_argument("--window-size=1920,1080")
    
    driver = uc.Chrome(options=options)
    driver.get(url)
    
    page_count = 1
    
    while True:
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.MuiGrid-item.css-mgehqq'))
            )
        except TimeoutException:
            print(f"Timed out waiting for page to load for ZIP {zip_code}")
            break
        
        # Scroll to load all agents
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        # Parse the page
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        agents = []
        
        for card in soup.select('div.MuiGrid-item.css-mgehqq'):
            try:
                name = card.select_one('h1.MuiTypography-subtitle2').text.strip()
                phone_div = card.find('div', {'data-testid': 'phoneNumber'})
                if phone_div:
                    phone = phone_div.find('a', {'data-testid': 'phoneLink'})
                    phone = phone.text.strip() if phone else "Phone not found"
                else:
                    phone = "Phone not found"
                
                agents.append({
                    'name': name,
                    'phone': phone,
                    'zip_code': zip_code
                })
            except Exception as e:
                print(f"Error parsing card: {e}")
                continue
        
        # Save agents to CSV immediately
        save_agents_to_csv(agents)
        
        print(f"Scraped page {page_count} for ZIP {zip_code} - {len(agents)} agents found and saved")
        
        # Pagination handling
        try:
            driver.execute_script("window.scrollTo(0, 0)")
            time.sleep(1)
            
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Go to next page"]')
            except NoSuchElementException:
                print(f"No more pages available for ZIP {zip_code}")
                break
                
            if "Mui-disabled" in next_button.get_attribute("class"):
                print(f"Reached last page for ZIP {zip_code}")
                break
                
            for attempt in range(3):
                try:
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    time.sleep(1)
                    next_button.click()
                    print("Clicked next button normally")
                    break
                except ElementClickInterceptedException:
                    driver.execute_script("arguments[0].click();", next_button)
                    print("Used JavaScript click fallback")
                    break
                except Exception as e:
                    if attempt == 2:
                        print("Failed to click next button after 3 attempts")
                        raise
                    print(f"Click attempt {attempt + 1} failed, retrying...")
                    time.sleep(2)
            
            try:
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.MuiGrid-item.css-mgehqq'))
                )
                page_count += 1
                time.sleep(2)
            except TimeoutException:
                print("New page didn't load properly")
                break
                
        except Exception as e:
            print(f"Navigation error: {str(e).split('Stacktrace:')[0]}")
            break
    
    driver.quit()

def scrape_multiple_zips(zip_codes):
    # Start with a fresh CSV file
    if os.path.exists(CSV_FILE):
        os.remove(CSV_FILE)
    for zip_code in zip_codes:
        scrape_coldwell_agents_for_zip(zip_code)
        print(f"\nFinished scraping ZIP {zip_code}")
        time.sleep(2)

if __name__ == "__main__":
    zip_codes = ['77494', '77449', '78660', '77433']
    print("Starting scraping for multiple ZIP codes...")
    scrape_multiple_zips(zip_codes)
    print(f"\nScraping completed. Data saved to {CSV_FILE}")


Starting scraping for multiple ZIP codes...

Starting scraping for ZIP code: 77494
Scraped page 1 for ZIP 77494 - 12 agents found and saved
Clicked next button normally
Scraped page 2 for ZIP 77494 - 12 agents found and saved
Clicked next button normally
Scraped page 3 for ZIP 77494 - 12 agents found and saved
Clicked next button normally
Scraped page 4 for ZIP 77494 - 12 agents found and saved
Clicked next button normally
Scraped page 5 for ZIP 77494 - 12 agents found and saved
Clicked next button normally
Scraped page 6 for ZIP 77494 - 12 agents found and saved
Clicked next button normally
Scraped page 7 for ZIP 77494 - 12 agents found and saved
Clicked next button normally
Scraped page 8 for ZIP 77494 - 12 agents found and saved
Clicked next button normally
Scraped page 9 for ZIP 77494 - 12 agents found and saved
Clicked next button normally
Scraped page 10 for ZIP 77494 - 12 agents found and saved
Clicked next button normally


KeyboardInterrupt: 

In [53]:
import time
import csv
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import (ElementClickInterceptedException, 
                                      TimeoutException,
                                      NoSuchElementException)

def scrape_coldwell_agents_for_zip(zip_code, csv_writer):
    url = f"https://www.coldwellbanker.com/zip/tx/{zip_code}/agents"
    print(f"\nStarting scraping for ZIP code: {zip_code}")
    
    # Set up undetected Chrome
    options = uc.ChromeOptions()
    options.headless = False  # Set to True if you don't want browser visible
    options.add_argument("--window-size=1920,1080")
    
    driver = uc.Chrome(options=options)
    driver.get(url)
    
    page_count = 1
    
    while True:
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.MuiGrid-item.css-mgehqq'))
            )
        except TimeoutException:
            print(f"Timed out waiting for page to load for ZIP {zip_code}")
            break
        
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        for card in soup.select('div.MuiGrid-item.css-mgehqq'):
            try:
                name = card.select_one('h1.MuiTypography-subtitle2').text.strip()
                
                phone_div = card.find('div', {'data-testid': 'phoneNumber'})
                if phone_div:
                    phone = phone_div.find('a', {'data-testid': 'phoneLink'})
                    phone = phone.text.strip() if phone else "Phone not found"
                else:
                    phone = "Phone not found"
                
                # Write agent to CSV immediately
                csv_writer.writerow({'name': name, 'phone': phone, 'zip_code': zip_code})
                print(f"Saved agent: {name} - {phone} (ZIP: {zip_code})")
                
            except Exception as e:
                print(f"Error parsing card: {e}")
                continue
            
        print(f"Scraped page {page_count} for ZIP {zip_code}")
            
        try:
            driver.execute_script("window.scrollTo(0, 0)")
            time.sleep(1)
            
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Go to next page"]')
            except NoSuchElementException:
                print(f"No more pages available for ZIP {zip_code}")
                break
                
            if "Mui-disabled" in next_button.get_attribute("class"):
                print(f"Reached last page for ZIP {zip_code}")
                break
                
            for attempt in range(3):
                try:
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    time.sleep(1)
                    next_button.click()
                    print("Clicked next button normally")
                    break
                except ElementClickInterceptedException:
                    driver.execute_script("arguments[0].click();", next_button)
                    print("Used JavaScript click fallback")
                    break
                except Exception as e:
                    if attempt == 2:
                        print("Failed to click next button after 3 attempts")
                        raise
                    print(f"Click attempt {attempt + 1} failed, retrying...")
                    time.sleep(2)
            
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.MuiGrid-item.css-mgehqq'))
            )
            page_count += 1
            time.sleep(2)
            
        except Exception as e:
            print(f"Navigation error: {str(e).split('Stacktrace:')[0]}")
            break
    
    driver.quit()

def scrape_multiple_zips(zip_codes):
    with open('coldwell_agents2.csv', mode='w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['name', 'phone', 'zip_code']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for zip_code in zip_codes:
            scrape_coldwell_agents_for_zip(zip_code, writer)
            time.sleep(2)  # Pause between ZIP codes

if __name__ == "__main__":
    zip_codes = [
        '78130',
        '75052', '78641', '77479', '78245', '77584', '78613', '78521', '78666', '75035', '78542',
        '75217', '77379', '79928', '76063', '77407', '77459', '78577', '79912', '78254', '76028',
        '78572', '75126', '76179', '77346', '76244', '77083', '77845', '75002', '77450', '75211',
        '78046', '78640', '77375', '77036', '75068', '77095', '77386', '75071', '75228', '75067',
        '77373', '75056', '78253', '75098', '78045', '77521', '77406', '77469', '75040', '78665',
        '75243', '75043', '77077', '78520', '75150', '78574', '78664', '77493', '78504', '78249',
        '78251', '78745', '79924', '75070', '78526', '76137', '78501', '77396', '75149', '77015',
        '77072', '78681', '78753', '78250', '77070', '78223', '77082', '75227', '78240', '78852',
        '77089', '78109', '76549', '75072', '75115', '75061', '77044', '76010', '78228', '76227',
        '77840', '75007', '78748', '75216', '78550', '78207', '77088', '77546', '76133', '76542',
        '78155', '75025', '76131', '75074', '77498', '76119', '77388', '75062', '75080', '75287',
        '79424', '77511', '78247', '78258', '78414', '77081', '78537', '75165', '76502', '78744',
        '75034', '78758', '76051', '78704', '77040', '75033', '77581', '75154', '76116', '75104',
        '78541', '77099', '78586', '77080', '77064', '75023', '75028', '75013', '78108', '79907',
        '78233', '75006', '77583', '77338', '75060', '77539', '78041', '78840', '76017', '75078',
        '75044', '76210', '78610', '75093', '77093', '75063', '77057', '77007', '78741', '77471',
        '75024', '78626', '77060', '75703', '78582', '78759', '76513', '78628', '76108', '75019',
        '76065', '77389', '79705', '75050', '76112', '79423', '75189', '78227', '77055', '78043',
        '77073', '76262', '78634', '78201', '77339', '78209', '79927', '77365', '78573', '76123',
        '79109', '77304', '78006'
    ]
    
    print("Starting scraping for multiple ZIP codes...")
    scrape_multiple_zips(zip_codes)


Starting scraping for multiple ZIP codes...

Starting scraping for ZIP code: 78130
Saved agent: Wendy Ague - (210) 865-7928 (ZIP: 78130)
Saved agent: Carrie Allen - (210) 834-3919 (ZIP: 78130)
Saved agent: Martin Arciba - (512) 949-0092 (ZIP: 78130)
Saved agent: Charles Borchers - (830) 237-8950 (ZIP: 78130)
Saved agent: Chris Bowers - (210) 643-9362 (ZIP: 78130)
Saved agent: Kimberly Brandsma - (210) 831-8288 (ZIP: 78130)
Saved agent: Aleida Castineira - (201) 785-6752 (ZIP: 78130)
Saved agent: Chase Collins - (281) 685-4257 (ZIP: 78130)
Saved agent: Cecy Crary - (619) 495-7971 (ZIP: 78130)
Saved agent: Laura D'Alfonso - (512) 853-0240 (ZIP: 78130)
Saved agent: Ken D'Alfonso - (808) 782-9113 (ZIP: 78130)
Saved agent: Dion Davis - (210) 845-2875 (ZIP: 78130)
Scraped page 1 for ZIP 78130
Clicked next button normally
Saved agent: Mary Evans - (512) 753-6311 (ZIP: 78130)
Saved agent: Shaila Galindo - (830) 423-7615 (ZIP: 78130)
Saved agent: Brenda Garza - (512) 214-6616 (ZIP: 78130)
Saved

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=135.0.7049.114)
Stacktrace:
	GetHandleVerifier [0x0032D363+60275]
	GetHandleVerifier [0x0032D3A4+60340]
	(No symbol) [0x001606F3]
	(No symbol) [0x0014FF20]
	(No symbol) [0x0016DDA2]
	(No symbol) [0x001D3D2F]
	(No symbol) [0x001EDFE9]
	(No symbol) [0x001CCE86]
	(No symbol) [0x0019C623]
	(No symbol) [0x0019D474]
	GetHandleVerifier [0x00578FE3+2467827]
	GetHandleVerifier [0x005745E6+2448886]
	GetHandleVerifier [0x0058F80C+2560028]
	GetHandleVerifier [0x00343DF5+153093]
	GetHandleVerifier [0x0034A3BD+179149]
	GetHandleVerifier [0x00334BB8+91080]
	GetHandleVerifier [0x00334D60+91504]
	GetHandleVerifier [0x0031FA10+4640]
	BaseThreadInitThunk [0x7718FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x773E82AE+286]
	RtlGetAppContainerNamedObjectPath [0x773E827E+238]
	(No symbol) [0x00000000]


In [43]:
import time
import csv
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import (ElementClickInterceptedException, 
                                      TimeoutException,
                                      NoSuchElementException)

def scrape_coldwell_agents_for_zip(zip_code, csv_writer):
    url = f"https://www.coldwellbanker.com/zip/tx/{zip_code}/agents"
    print(f"\nStarting scraping for ZIP code: {zip_code}")
    
    # Set up undetected Chrome with more options
    options = uc.ChromeOptions()
    options.headless = False  # Let's see the browser first
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    
    # Enable logging to see what's happening
    options.set_capability("goog:loggingPrefs", {'performance': 'ALL'})
    
    driver = uc.Chrome(
        options=options,
        version_main=114  # Match your Chrome version
    )
    
    try:
        print("Opening browser...")
        driver.get(url)
        print(f"Loaded URL: {url}")
        
        # Let's wait and see what's on the page
        input("Press Enter after you verify the page loaded in browser...")
        
        # Check if we got blocked
        page_source = driver.page_source
        if "access denied" in page_source.lower():
            print("WEBSITE BLOCKED US!")
            return
        
        # Try to find at least one agent card
        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.MuiGrid-item'))
            )
            print("Found agent cards!")
        except TimeoutException:
            print("Couldn't find any agent cards. Page structure might have changed.")
            print("Current page title:", driver.title)
            with open(f"debug_page_{zip_code}.html", "w", encoding="utf-8") as f:
                f.write(driver.page_source)
            print("Saved page source for debugging")
            return
        
        # Rest of your scraping logic...
        # For now let's just print the first agent's name
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        first_name = soup.select_one('h1.MuiTypography-subtitle2')
        if first_name:
            print("First agent found:", first_name.text.strip())
        else:
            print("No agents found on page")
            
    except Exception as e:
        print(f"Error: {str(e)}")
    finally:
        input("Press Enter to close browser...")
        driver.quit()

def scrape_multiple_zips(zip_codes):
    with open('coldwell_agents.csv', mode='w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['name', 'phone', 'zip_code']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        # Just test with first ZIP code for now
        scrape_coldwell_agents_for_zip(zip_codes[0], writer)

if __name__ == "__main__":
    test_zip_codes = ['77494']  # Just one ZIP to test
    print("Starting test scraping...")
    scrape_multiple_zips(test_zip_codes)

Starting test scraping...

Starting scraping for ZIP code: 77494
Opening browser...
Loaded URL: https://www.coldwellbanker.com/zip/tx/77494/agents


Press Enter after you verify the page loaded in browser... 


Found agent cards!
First agent found: Burke Abercrombie


Press Enter to close browser... 


In [1]:
import time
import csv
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import (ElementClickInterceptedException, 
                                      TimeoutException,
                                      NoSuchElementException)

def scrape_coldwell_agents_for_zip(zip_code, csv_writer):
    url = f"https://www.coldwellbanker.com/zip/ca/{zip_code}/agents"
    print(f"\nStarting scraping for ZIP code: {zip_code}")
    
    # Set up undetected Chrome
    options = uc.ChromeOptions()
    options.headless = False  # Set to True if you don't want browser visible
    options.add_argument("--window-size=1920,1080")
    
    driver = uc.Chrome(options=options)
    driver.get(url)
    
    page_count = 1
    
    while True:
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.MuiGrid-item.css-mgehqq'))
            )
        except TimeoutException:
            print(f"Timed out waiting for page to load for ZIP {zip_code}")
            break
        
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        for card in soup.select('div.MuiGrid-item.css-mgehqq'):
            try:
                name = card.select_one('h1.MuiTypography-subtitle2').text.strip()
                
                phone_div = card.find('div', {'data-testid': 'phoneNumber'})
                if phone_div:
                    phone = phone_div.find('a', {'data-testid': 'phoneLink'})
                    phone = phone.text.strip() if phone else "Phone not found"
                else:
                    phone = "Phone not found"
                
                # Write agent to CSV immediately
                csv_writer.writerow({'name': name, 'phone': phone, 'zip_code': zip_code})
                print(f"Saved agent: {name} - {phone} (ZIP: {zip_code})")
                
            except Exception as e:
                print(f"Error parsing card: {e}")
                continue
            
        print(f"Scraped page {page_count} for ZIP {zip_code}")
            
        try:
            driver.execute_script("window.scrollTo(0, 0)")
            time.sleep(1)
            
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Go to next page"]')
            except NoSuchElementException:
                print(f"No more pages available for ZIP {zip_code}")
                break
                
            if "Mui-disabled" in next_button.get_attribute("class"):
                print(f"Reached last page for ZIP {zip_code}")
                break
                
            for attempt in range(3):
                try:
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    time.sleep(1)
                    next_button.click()
                    print("Clicked next button normally")
                    break
                except ElementClickInterceptedException:
                    driver.execute_script("arguments[0].click();", next_button)
                    print("Used JavaScript click fallback")
                    break
                except Exception as e:
                    if attempt == 2:
                        print("Failed to click next button after 3 attempts")
                        raise
                    print(f"Click attempt {attempt + 1} failed, retrying...")
                    time.sleep(2)
            
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.MuiGrid-item.css-mgehqq'))
            )
            page_count += 1
            time.sleep(2)
            
        except Exception as e:
            print(f"Navigation error: {str(e).split('Stacktrace:')[0]}")
            break
    
    driver.quit()

def scrape_multiple_zips(zip_codes):
    with open('coldwell_agents_ca.csv', mode='w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['name', 'phone', 'zip_code']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for zip_code in zip_codes:
            scrape_coldwell_agents_for_zip(zip_code, writer)
            time.sleep(2)  # Pause between ZIP codes

if __name__ == "__main__":
    zip_codes = [
        '90011', '90650', '91331', '90201', '90044', '91342', '92335', '91744', 
        '90003', '92804', '90250', '91706', '93550', '91709', '92336', '92503', 
        '90706', '91710', '90280', '92509', '92805', '91741', '92553', '92376', 
        '92563', '91766', '92345', '92562', '92392', '91765', '92346', '91761', 
        '91764', '91762', '92504', '91763', '92399', '91767', '92324', '92801', 
        '92394', '92377', '91770', '92325', '91768', '91773', '92378', '91776', 
        '91775', '92382', '91786', '92327', '91790', '92335', '91791', '92337', 
        '91792', '92338', '91793', '92339', '91789', '92342', '92347', '92350', 
        '92354', '92356', '92358', '92359', '92364', '92365', '92368', '92371', 
        '92372', '92373', '92374', '92381', '92385', '92386', '92395', '92397', 
        '92401', '92404', '92405', '92407', '92408', '92410', '92411', '92501', 
        '92505', '92507', '92508', '92530', '92532', '92543', '92544', '92545', 
        '92548', '92549', '92551', '92557', '92561', '92564', '92567', '92570', 
        '92571', '92572', '92581', '92582', '92583', '92584', '92585', '92586', 
        '92587', '92589', '92590', '92591', '92592', '92593', '92595', '92596', 
        '92599', '92602', '92603', '92604', '92606', '92610', '92612', '92614', 
        '92617', '92618', '92620', '92624', '92626', '92627', '92629', '92630', 
        '92637', '92646', '92647', '92648', '92649', '92651', '92653', '92655', 
        '92656', '92657', '92660', '92661', '92662', '92663', '92672', '92673', 
        '92675', '92677', '92679', '92683', '92688', '92691', '92692', '92694', 
        '92701', '92703', '92704', '92705', '92706', '92707', '92708', '92780', 
        '92782', '92799', '92801', '92802', '92804', '92805', '92806', '92807', 
        '92808', '92821', '92823', '92831', '92832', '92833', '92835', '92840', 
        '92841', '92843', '92844', '92845', '92861', '92865', '92866', '92867', 
        '92868', '92869', '92870', '92871', '92877', '92878', '92879', '92880', 
        '92881', '92882', '92883', '92886', '92887', '92899', '93010', '93012', 
        '93013', '93015', '93021', '93022', '93023', '93030', '93033', '93035', 
        '93036', '93041', '93042', '93043', '93044', '93060', '93063', '93065', 
        '93066', '93067', '93094', '93099', '93101', '93103', '93105', '93110', 
        '93111', '93117', '93130', '93140', '93150', '93160', '93190', '93199', 
        '93215', '93221', '93226', '93230', '93241', '93242', '93243', '93245', 
        '93249', '93250', '93251', '93252', '93254', '93255', '93256', '93257', 
        '93258', '93260', '93262', '93263', '93265', '93266', '93267', '93268', 
        '93270', '93271', '93272', '93274', '93275', '93276', '93277', '93278', 
        '93279', '93280', '93282', '93283', '93285', '93286', '93287', '93290', 
        '93291', '93292', '93301', '93304', '93305', '93306', '93307', '93308', 
        '93309', '93311', '93312', '93313', '93314', '93401', '93402', '93405', 
        '93410', '93420', '93422', '93426', '93427', '93428', '93430', '93432', 
        '93433', '93434', '93436', '93437', '93442', '93444', '93445', '93446', 
        '93449', '93450', '93451', '93452', '93453', '93454', '93455', '93458', 
        '93460', '93461', '93463', '93464', '93465', '93475', '93483', '93510', 
        '93516', '93523', '93527', '93528', '93532', '93534', '93535', '93536', 
        '93539', '93541', '93543', '93544', '93545', '93546', '93550', '93551', 
        '93552', '93553', '93554', '93555', '93556', '93560', '93561', '93562', 
        '93563', '93581', '93584', '93586', '93590', '93591', '93592', '93596', 
        '93599', '93610', '93612', '93614', '93615', '93616', '93618', '93619', 
        '93620', '93621', '93622', '93623', '93624', '93625', '93626', '93627', 
        '93630', '93631', '93635', '93637', '93638', '93640', '93641', '93643', 
        '93644', '93645', '93646', '93647', '93648', '93649', '93650', '93651', 
        '93652', '93653', '93654', '93656', '93657', '93660', '93661', '93662', 
        '93664', '93665', '93666', '93667', '93668', '93669', '93670', '93673', 
        '93675', '93701', '93702', '93703', '93704', '93705', '93706', '93710', 
        '93711', '93720', '93721', '93722', '93725', '93726', '93727', '93728', 
        '93729', '93730', '93737', '93740', '93741', '93744', '93745', '93747', 
        '93750', '93755', '93760', '93761', '93764', '93765', '93771', '93772', 
        '93773', '93774', '93775', '93776', '93777', '93778', '93779', '93786', 
        '93790', '93791', '93792', '93793', '93794', '93844', '93888', '93901', 
        '93905', '93906', '93907', '93908', '93912', '93915', '93920', '93921', 
        '93923', '93924', '93925', '93926', '93927', '93928', '93930', '93932', 
        '93933', '93940', '93942', '93943', '93944', '93950', '93953', '93954', 
        '93955', '93960', '93962', '94002', '94005', '94010', '94011', '94014', 
        '94015', '94019', '94020', '94021', '94022', '94024', '94025', '94027', 
        '94028', '94030', '94035', '94037', '94038', '94039', '94040', '94041', 
        '94043', '94044', '94060', '94061', '94062', '94063', '94064', '94065', 
        '94066', '94070', '94074', '94080', '94083', '94085', '94086', '94087', 
        '94089', '94096', '94098', '94102', '94103', '94104', '94105', '94107', 
        '94108', '94109', '94110', '94111', '94112', '94114', '94115', '94116', 
        '94117', '94118', '94119', '94120', '94121', '94122', '94123', '94124', 
        '94125', '94126', '94127', '94128', '94129', '94130', '94131', '94132', 
        '94133', '94134', '94137', '94139', '94140', '94141', '94142', '94143', 
        '94144', '94145', '94146', '94147', '94151', '94158', '94159', '94160', 
        '94161', '94163', '94164', '94172', '94177', '94188', '94203', '94204', 
        '94205', '94206', '94207', '94208', '94209', '94211', '94229', '94230', 
        '94232', '94234', '94235', '94236', '94237', '94239', '94240', '94244', 
        '94245', '94247', '94248', '94249', '94250', '94252', '94254', '94256', 
        '94257', '94258', '94259', '94261', '94262', '94263', '94267', '94268', 
        '94269', '94271', '94273', '94274', '94277', '94278', '94279', '94280', 
        '94282', '94283', '94284', '94285', '94286', '94287', '94288', '94289', 
        '94290', '94291', '94293', '94294', '94295', '94296', '94297', '94298', 
        '94299', '94301', '94302', '94303', '94304', '94305', '94306', '94309', 
        '94401', '94402', '94403', '94404', '94497', '94501', '94502', '94505', 
        '94506', '94507', '94509', '94510', '94512', '94513', '94514', '94515', 
        '94517', '94518', '94519', '94520', '94521', '94522', '94523', '94525', 
        '94526', '94527', '94528', '94529', '94530', '94531', '94533', '94534', 
        '94535', '94536', '94537', '94538', '94539', '94540', '94541', '94542', 
        '94544', '94545', '94546', '94547', '94548', '94549', '94550', '94551', 
        '94552', '94553', '94555', '94556', '94557', '94558', '94559', '94560', 
        '94561', '94563', '94564', '94565', '94566', '94567', '94568', '94569', 
        '94570', '94571', '94572', '94573', '94574', '94575', '94576', '94577', 
        '94578', '94579', '94580', '94581', '94582', '94583', '94585', '94586', 
        '94587', '94588', '94589', '94590', '94591', '94592', '94595', '94596', 
        '94597', '94598', '94599', '94601', '94602', '94603', '94604', '94605', 
        '94606', '94607', '94608', '94609', '94610', '94611', '94612', '94613', 
        '94614', '94615', '94617', '94618', '94619', '94620', '94621', '94622', 
        '94623', '94624', '94625', '94649', '94659', '94660', '94661', '94662', 
        '94666', '94701', '94702', '94703', '94704', '94705', '94706', '94707', 
        '94708', '94709', '94710', '94712', '94720', '94801', '94802', '94803', 
        '94804', '94805', '94806', '94807', '94808', '94820', '94850', '94901', 
        '94903', '94904', '94912', '94913', '94914', '94915', '94920', '94922', 
        '94923', '94924', '94925', '94928', '94929', '94930', '94931', '94933', 
        '94937', '94938', '94939', '94940', '94941', '94942', '94945', '94946', 
        '94947', '94948', '94949', '94950', '94951', '94952', '94953', '94954', 
        '94955', '94956', '94957', '94960', '94963', '94964', '94965', '94966', 
        '94970', '94971', '94972', '94973', '94974', '94975', '94976', '94977', 
        '94978', '94979', '94998', '94999', '95002', '95003', '95004', '95005', 
        '95006', '95007', '95008', '95009', '95010', '95011', '95012', '95013', 
        '95014', '95015', '95017', '95018', '95019', '95020', '95021', '95023', 
        '95024', '95026', '95030', '95031', '95032', '95033', '95035', '95036', 
        '95037', '95038', '95039', '95041', '95042', '95043', '95044', '95045', 
        '95046', '95050', '95051', '95052', '95053', '95054', '95055', '95056', 
        '95060', '95061', '95062', '95063', '95064', '95065', '95066', '95067', 
        '95070', '95071', '95073', '95075', '95076', '95077', '95101', '95103', 
        '95106', '95108', '95109', '95110', '95111', '95112', '95113', '95115', 
        '95116', '95117', '95118', '95119', '95120', '95121', '95122', '95123', 
        '95124', '95125', '95126', '95127', '95128', '95129', '95130', '95131', 
        '95132', '95133', '95134', '95135', '95136', '95138', '95139', '95140', 
        '95141', '95148', '95150', '95151', '95152', '95153', '95154', '95155', 
        '95156', '95157', '95158', '95159', '95160', '95161', '95164', '95170', 
        '95172', '95173', '95190', '95191', '95192', '95193', '95194', '95196', 
        '95202', '95203', '95204', '95205', '95206', '95207', '95208', '95209', 
        '95210', '95211', '95212', '95213', '95215', '95219', '95220', '95221', 
        '95222', '95223', '95224', '95225', '95226', '95227', '95228', '95229', 
        '95230', '95231', '95232', '95233', '95234', '95236', '95237', '95240', 
        '95241', '95242', '95245', '95246', '95247', '95248', '95249', '95251', 
        '95252', '95253', '95254', '95255', '95257', '95258', '95267', '95269', 
        '95296', '95297', '95301', '95303', '95304', '95305', '95306', '95307', 
        '95309', '95310', '95311', '95312', '95313', '95315', '95316', '95317', 
        '95318', '95319', '95320', '95321', '95322', '95323', '95324', '95325', 
        '95326', '95327', '95328', '95329', '95330', '95333', '95334', '95335', 
        '95336', '95337', '95338', '95340', '95341', '95343', '95344', '95345', 
        '95346', '95347', '95348', '95350', '95351', '95352', '95353', '95354', 
        '95355', '95356', '95357', '95358', '95360', '95361', '95363', '95364', 
        '95365', '95366', '95367', '95368', '95369', '95370', '95372', '95373', 
        '95374', '95375', '95376', '95377', '95378', '95379', '95380', '95381', 
        '95382', '95383', '95385', '95386', '95387', '95388', '95389', '95391', 
        '95397', '95398', '95401', '95403', '95404', '95405', '95407', '95409', 
        '95410', '95412', '95415', '95416', '95417', '95419', '95420', '95421', 
        '95422', '95423', '95425', '95426', '95427', '95428', '95429', '95430', 
        '95431', '95432', '95433', '95435', '95436', '95437', '95439', '95441', 
        '95442', '95443', '95444', '95445', '95446', '95448', '95449', '95450', 
        '95451', '95452', '95453', '95454', '95456', '95457', '95458', '95459', 
        '95460', '95461', '95462', '95463', '95464', '95465', '95466', '95467', 
        '95468', '95469', '95470', '95471', '95472', '95473', '95476', '95480', 
        '95482', '95485', '95486', '95487', '95488', '95490', '95492', '95493', 
        '95494', '95497', '95501', '95503', '95511', '95514', '95519', '95521', 
        '95524', '95525', '95526', '95527', '95528', '95531', '95532', '95534', 
        '95536', '95537', '95538', '95540', '95542', '95543', '95545', '95546', 
        '95547', '95548', '95549', '95550', '95551', '95552', '95553', '95554', 
        '95555', '95556', '95558', '95559', '95560', '95562', '95563', '95564', 
        '95565', '95567', '95568', '95569', '95570', '95571', '95573', '95585', 
        '95587', '95589', '95595', '95602', '95603', '95604', '95605', '95606', 
        '95607', '95608', '95609', '95610', '95611', '95612', '95613', '95614', 
        '95615', '95616', '95617', '95618', '95619', '95620', '95621', '95623', 
        '95624', '95625', '95626', '95627', '95628', '95629', '95630', '95631', 
        '95632', '95633', '95634', '95635', '95636', '95637', '95638', '95639', 
        '95640', '95641', '95642', '95644', '95645', '95646', '95648', '95650', 
        '95651', '95652', '95653', '95654', '95655', '95656', '95658', '95659', 
        '95660', '95661', '95662', '95663', '95664', '95665', '95666', '95667', 
        '95668', '95669', '95670', '95671', '95672', '95673', '95674', '95675', 
        '95676', '95677', '95678', '95679', '95680', '95681', '95682', '95683', 
        '95684', '95685', '95686', '95687', '95688', '95689', '95690', '95691', 
        '95692', '95693', '95694', '95695', '95696', '95697', '95698', '95699', 
        '95701', '95703', '95709', '95712', '95713', '95714', '95715', '95717', 
        '95720', '95721', '95722', '95724', '95726', '95728', '95735', '95736', 
        '95741', '95742', '95746', '95747', '95757', '95758', '95759', '95762', 
        '95763', '95765', '95776', '95798', '95799', '95811', '95814', '95815', 
        '95816', '95817', '95818', '95819', '95820', '95821', '95822', '95823', 
        '95824', '95825', '95826', '95827', '95828', '95829', '95830', '95831', 
        '95832', '95833', '95834', '95835', '95836', '95837', '95838', '95840', 
        '95841', '95842', '95843', '95851', '95852', '95853', '95860', '95864', 
        '95865', '95866', '95867', '95894', '95899', '95901', '95903', '95910', 
        '95912', '95914', '95915', '95916', '95917', '95918', '95919', '95920', 
        '95922', '95923', '95924', '95925', '95926', '95927', '95928', '95929', 
        '95930', '95932', '95934', '95935', '95936', '95937', '95938', '95939', 
        '95940', '95941', '95942', '95943', '95944', '95945', '95946', '95947', 
        '95948', '95949', '95950', '95951', '95953', '95954', '95955', '95956', 
        '95957', '95958', '95959', '95960', '95961', '95962', '95963', '95965', 
        '95966', '95967', '95968', '95969', '95970', '95971', '95972', '95973', 
        '95974', '95975', '95976', '95977', '95978', '95979', '95980', '95981', 
        '95982', '95983', '95984', '95986', '95987', '95988', '95991', '95992', 
        '95993', '96001', '96002', '96003', '96006', '96007', '96008', '96009', 
        '96010', '96011', '96013', '96014', '96015', '96016', '96017', '96019', 
        '96020', '96021', '96022', '96023', '96024', '96025', '96027', '96028', 
        '96029', '96031', '96032', '96033', '96034', '96035', '96037', '96038', 
        '96039', '96040', '96041', '96044', '96045', '96046', '96047', '96048', 
        '96049', '96050', '96051', '96052', '96054', '96055', '96056', '96057', 
        '96058', '96059', '96061', '96062', '96063', '96064', '96065', '96067', 
        '96068', '96069', '96070', '96071', '96073', '96074', '96075', '96076', 
        '96078', '96079', '96080', '96084', '96085', '96086', '96087', '96088', 
        '96089', '96090', '96091', '96092', '96093', '96094', '96095', '96096', 
        '96097', '96099', '96101', '96103', '96104', '96105', '96106', '96107', 
        '96108', '96109', '96110', '96111', '96112', '96113', '96114', '96115', 
        '96116', '96117', '96118', '96119', '96120', '96121', '96122', '96123', 
        '96124', '96125', '96126', '96127', '96128', '96129', '96130', '96132', 
        '96133', '96134', '96135', '96136', '96137', '96140', '96141', '96142', 
        '96143', '96145', '96146', '96148', '96150', '96151', '96152', '96154', 
        '96155', '96156', '96157', '96158', '96160', '96161', '96162'
    ]
    
    print("Starting scraping for California ZIP codes...")
    scrape_multiple_zips(zip_codes)

Starting scraping for California ZIP codes...

Starting scraping for ZIP code: 90011
Saved agent: Matthew Abraham - (323) 449-7840 (ZIP: 90011)
Saved agent: Heidi Adams - (818) 461-2278 (ZIP: 90011)
Saved agent: Paul Adams - (301) 906-6252 (ZIP: 90011)
Saved agent: Neal Adler - (818) 787-1460 (ZIP: 90011)
Saved agent: Barbara Adler - (201) 562-9727 (ZIP: 90011)
Saved agent: Jeff Adler - (818) 489-6533 (ZIP: 90011)
Saved agent: Mystoura Afolabi - (310) 409-5650 (ZIP: 90011)
Saved agent: Irish Africano - (818) 487-5872 (ZIP: 90011)
Saved agent: Aram Afshar - (310) 385-3174 (ZIP: 90011)
Saved agent: Philomena Agege - (310) 701-3572 (ZIP: 90011)
Saved agent: Nestor Aguilar - (818) 518-0171 (ZIP: 90011)
Saved agent: June Ahn - (323) 855-5558 (ZIP: 90011)
Scraped page 1 for ZIP 90011
Clicked next button normally
Saved agent: Elinor Ahoubim - (818) 861-4583 (ZIP: 90011)
Saved agent: Jade Ajose - (469) 879-9928 (ZIP: 90011)
Saved agent: Thais Alberto - (818) 572-6589 (ZIP: 90011)
Saved agent: 

KeyboardInterrupt: 

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
from urllib.parse import quote_plus
import pandas as pd

class OffMarketPropertyScraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        self.base_google_url = "https://www.google.com/search?q="
        self.results = []
        
    def build_search_queries(self, locations):
        """Generate search queries for different locations"""
        queries = []
        base_queries = [
            "off market properties in",
            "pocket listings in",
            "pre-market homes in",
            "coming soon properties in",
            "wholesale properties in",
            "real estate investors off market in"
        ]
        
        for location in locations:
            for query in base_queries:
                queries.append(f"{query} {location}")
                
        return queries
    
    def scrape_google_results(self, query, pages=1):
        """Scrape Google search results for a given query"""
        url = self.base_google_url + quote_plus(query)
        
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extract organic results
            for result in soup.select('div.g'):
                link = result.select_one('a[href^="http"]')
                if link:
                    title = link.text
                    url = link['href']
                    description = result.select_one('div.IsZvec')
                    description = description.text if description else ""
                    
                    # Check if this looks like a property listing
                    if self.is_property_related(title, description):
                        self.results.append({
                            'title': title,
                            'url': url,
                            'description': description,
                            'source': 'Google Search',
                            'query': query
                        })
            
            # TODO: Add pagination handling
            
        except Exception as e:
            print(f"Error scraping {query}: {e}")
    
    def scrape_google_maps(self, location):
        """Scrape potential FSBO listings from Google Maps"""
        maps_url = f"https://www.google.com/maps/search/for+sale+by+owner+in+{quote_plus(location)}"
        
        try:
            response = requests.get(maps_url, headers=self.headers)
            response.raise_for_status()
            
            # Note: Google Maps scraping is more complex and may require Selenium
            # This is a placeholder for the concept
            # Actual implementation would parse business listings with FSBO signs
            
        except Exception as e:
            print(f"Error scraping Google Maps: {e}")
    
    def is_property_related(self, title, description):
        """Determine if a result appears to be property-related"""
        property_terms = ['property', 'home', 'house', 'real estate', 'listing', 
                         'investment', 'wholesale', 'off market', 'pocket listing']
        combined_text = f"{title} {description}".lower()
        return any(term in combined_text for term in property_terms)
    
    def extract_contacts(self, text):
        """Extract potential phone numbers and emails from text"""
        phones = re.findall(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', text)
        emails = re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', text)
        return phones, emails
    
    def run(self, locations, pages_per_query=1):
        """Main execution method"""
        queries = self.build_search_queries(locations)
        
        for query in queries:
            print(f"Processing query: {query}")
            self.scrape_google_results(query, pages=pages_per_query)
            time.sleep(2)  # Be polite with delay
            
        # Additional scraping could go here
        # self.scrape_google_maps(locations[0])
        
        return pd.DataFrame(self.results)

# Example Usage
if __name__ == "__main__":
    scraper = OffMarketPropertyScraper()
    locations = ["Los Angeles CA", "Miami FL", "New York NY"]  # Add your target locations
    results_df = scraper.run(locations)
    results_df.to_csv('off_market_properties.csv', index=False)

Processing query: off market properties in Los Angeles CA
Processing query: pocket listings in Los Angeles CA
Processing query: pre-market homes in Los Angeles CA
Processing query: coming soon properties in Los Angeles CA
Processing query: wholesale properties in Los Angeles CA
Processing query: real estate investors off market in Los Angeles CA
Processing query: off market properties in Miami FL
Processing query: pocket listings in Miami FL
Processing query: pre-market homes in Miami FL
Processing query: coming soon properties in Miami FL
Processing query: wholesale properties in Miami FL
Processing query: real estate investors off market in Miami FL
Processing query: off market properties in New York NY
Processing query: pocket listings in New York NY
Processing query: pre-market homes in New York NY
Processing query: coming soon properties in New York NY
Processing query: wholesale properties in New York NY
Processing query: real estate investors off market in New York NY
